In [ ]:
# # Copy notebook from Google Drive to working directory
!cp -r "/content/drive/MyDrive/2 2568Machine Learning & Predictive Analytics/"* .

# # # Add notebook to Git
### !git add rfm_behaver_online_retail.ipynb
!git add .
!git commit -m "feat: add RFM notebook"
!git push

In [ ]:
import kagglehub
import pandas as pd
import os
import numpy as np

# 1. Download dataset
path = kagglehub.dataset_download("sanlian/online-retail-dataset")
print("Dataset folder:", path)

# 2. List files inside dataset
print("Files:", os.listdir(path))

# 3. Read file
file_path = os.path.join(path, "online_retail_II.xlsx")
# df = pd.read_excel(file_path)

df1 = pd.read_excel(file_path, sheet_name="Year 2009-2010")
df2 = pd.read_excel(file_path, sheet_name="Year 2010-2011")

df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
df1.shape

In [ ]:
df2.shape

In [ ]:
df.shape

In [ ]:
print(df.info())

In [ ]:
df1.head(5)

In [ ]:
df2.head(5)

In [ ]:
df.head(10)

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

ตรวจสอบ Min Max ว่ามีค่าติดลบ หรือ ผิดปกติไหม

In [ ]:
df.describe().loc[['min', 'max']]

Clean data before using

In [ ]:
df = df[df['Customer ID'].notna()]  # ลบข้อมูลที่ไม่มี CustomerID ถ้าไม่มี customer ID เราก็ไม่สามารถรู้ได้ว่า เป็นลูกค้าคนเดียวกันหรือไม่

df = df[df['Quantity'] > 0] # ลบการคืนสินค้า

df = df[df['Price'] > 0]  # ลบราคาที่ผิดปกติของแถม

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])# แปลงวันที่เป็น datetime

ตรวจสอบ Outlier ก่อนนำไปใช้ scaling min max เพื่อป้องกันค่าที่ผิดปกติ
max	10,953.50 Outlier

In [ ]:
df['Price'].describe(percentiles=[0.01, 0.05, 0.40, 0.70, 0.95, 0.99])

ใช้ Log Transformation เพื่อป้องกันข้อมูล Outlier

In [ ]:
# df['Price_log'] = np.log1p(df['Price'])

In [ ]:
# df['Price_log'].describe(percentiles=[0.01, 0.05, 0.95, 0.99])

In [ ]:
df['Quantity'].describe(percentiles=[0.01, 0.05, 0.40, 0.70, 0.95, 0.99])

In [ ]:
# df['Quantity_log']= np.log1p(df['Quantity'])

In [ ]:
# df['Quantity_log'].describe(percentiles=[0.01, 0.05, 0.40, 0.70, 0.95, 0.99])

In [ ]:
df['TotalPrice'] = df['Quantity'] * df['Price']

In [ ]:
df['TotalPrice'].describe(percentiles=[0.01, 0.05, 0.40, 0.70, 0.95, 0.99])

กำหนดวันตัดเวลา (70% แรกเป็นอดีต, 30% หลังเป็นอนาคต)

In [ ]:
cutoff_date = df["InvoiceDate"].quantile(0.7)
df_past = df[df["InvoiceDate"] <= cutoff_date]
df_future = df[df["InvoiceDate"] > cutoff_date]
print("cutoff_date:", cutoff_date)

df_future

RFM จากอดีต (เหมือนเดิม แต่ใช้ df_past)

กำหนด “วันอ้างอิง” (reference date) เพื่อใช้วัดว่า ลูกค้าซื้อครั้งล่าสุด + 1 วัน

Recency               = today − max(InvoiceDate) = วันที่ซื้อครั้งล่าสุด
ความหมาย
ระยะเวลาตั้งแต่ลูกค้าซื้อครั้งล่าสุด
การตีความ
ค่าน้อย → ลูกค้าเพิ่งซื้อ → active
ค่าสูง → ลูกค้าไม่ได้ซื้อมานาน → เสี่ยง churn
บทบาท
ตัวชี้วัดความสดของความสัมพันธ์

Frequency             = distinct Invoice = จำนวนใบ Invoice ที่ไม่ซ้ำ
ความหมาย
ความถี่ในการซื้อของลูกค้า
การตีความ
ค่าสูง → ซื้อบ่อย → engagement สูง
ค่าต่ำ → ซื้อไม่บ่อย
บทบาท
สะท้อนพฤติกรรมการกลับมาซื้อซ้ำ

Monetary              = sum(Sales) = ยอดใช้จ่ายรวมทั้งหมด
ความหมาย
มูลค่ารวมที่ลูกค้าใช้จ่ายกับธุรกิจ
การตีความ
ค่าสูง → ลูกค้ามูลค่าสูง (high value)
ค่าต่ำ → ลูกค้ามูลค่าต่ำ
บทบาท
ตัวชี้วัดคุณค่าทางรายได้

Tenure                = today − min(InvoiceDate) = วันที่ซื้อครั้งแรก
ความหมาย
ระยะเวลาที่ลูกค้าเริ่มมีความสัมพันธ์กับธุรกิจ
การตีความ
ค่าสูง → ลูกค้าเก่า / long-term
ค่าต่ำ → ลูกค้าใหม่
บทบาท
แยกลูกค้าใหม่กับลูกค้าเดิม
เสริม Recency (มองคนละมุม)

Avg_Order_Value       = Monetary / Frequency
ความหมาย
มูลค่าเฉลี่ยต่อการซื้อหนึ่งครั้ง
การตีความ
ค่าสูง → ซื้อไม่บ่อยแต่ใช้เงินต่อครั้งสูง
ค่าต่ำ → ซื้อบ่อยแต่ยอดต่อครั้งต่ำ
บทบาท
อธิบายรูปแบบการใช้จ่ายที่ Monetary เพียงอย่างเดียวไม่เห็น

Product_Diversity     = unique StockCode = จำนวนสินค้า (StockCode) ที่ไม่ซ้ำ
ความหมาย
ความหลากหลายของสินค้าที่ลูกค้าซื้อ
การตีความ
ค่าสูง → exploratory / engagement สูง
ค่าต่ำ → ซื้อซ้ำสินค้าเดิม (loyal ต่อ product)
บทบาท
วัดระดับการมีส่วนร่วมกับ assortment สินค้า

Purchase_Regularity = ส่วนเบี่ยงเบนมาตรฐานของช่วงเวลาระหว่างการซื้อ
ความหมาย
ความสม่ำเสมอของการซื้อ
การตีความ
ค่าต่ำ → ซื้อสม่ำเสมอ
ค่าสูง → ซื้อไม่แน่นอน
บทบาท
สะท้อน temporal behavior
ใช้เสริมในเชิงวิเคราะห์ (optional)

In [ ]:
reference_date = cutoff_date + pd.Timedelta(days=1)

rfm_past = df_past.groupby("Customer ID").agg({
    "InvoiceDate": lambda x: (reference_date - x.max()).days,
    "Invoice": "nunique",
    "TotalPrice": "sum"
}).reset_index()

rfm_past.columns = ["Customer ID", "Recency", "Frequency", "Monetary"]

tenure = df_past.groupby("Customer ID")["InvoiceDate"] \
                .min() \
                .apply(lambda x: (reference_date - x).days)

aov = rfm_past["Monetary"] / rfm_past["Frequency"]

product_diversity = df_past.groupby("Customer ID")["StockCode"].nunique()

def purchase_regularity(x):
    x = x.sort_values()
    if len(x) < 3:
        return 0
    return x.diff().dt.days.std()

regularity = df_past.groupby("Customer ID")["InvoiceDate"] \
                    .apply(purchase_regularity) # Purchase Regularity (std of days between purchases)

rfm_past["Tenure"] = tenure.values
rfm_past["Avg_Order_Value"] = aov
rfm_past["Product_Diversity"] = product_diversity.values
rfm_past["Purchase_Regularity"] = regularity.values

Target = repurchase from future data set by count distinct invoice

In [ ]:
repurchase_count = df_future.groupby("Customer ID")["Invoice"].nunique()

rfm_past["Repurchase_Count"] = (
    rfm_past["Customer ID"].map(repurchase_count).fillna(0)
)
rfm_past

กำหนด “วันอ้างอิง” (reference date)
เพื่อใช้วัดว่า ลูกค้าซื้อครั้งล่าสุด + 1 วัน

In [ ]:
rfm_past.describe().loc[['min', 'max']]

In [ ]:
rfm_past[rfm_past['Monetary'] > 18741]

แต่ละตัวมี “Scale ไม่เท่ากัน” เช่น:
Recency = หลักร้อย
Frequency = หลักร้อย
Monetary = หลักหมื่น
โมเดลจะ ถูกค่ามากกว่ามาก
ดังนั้นต้องปรับให้อยู่ในช่วงเท่า ๆ กัน:

โดยใช้ log1p เปลี่ยนรูปทรงข้อมูลแต่ไม่ fix ช่วงค่า

In [ ]:
rfm_past['Recency_log'] = np.log1p(rfm_past['Recency'])
rfm_past['Frequency_log'] = np.log1p(rfm_past['Frequency'])
rfm_past['Monetary_log'] = np.log1p(rfm_past['Monetary'])
rfm_past['Tenure_log'] = np.log1p(rfm_past['Tenure'])
rfm_past['Avg_Order_Value_log'] = np.log1p(rfm_past['Avg_Order_Value'])
rfm_past['Product_Diversity_log'] = np.log1p(rfm_past['Product_Diversity'])
rfm_past['Purchase_Regularity_log'] = np.log1p(rfm_past['Purchase_Regularity'])

In [ ]:
rfm_past.describe().loc[['min', 'max']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

rfm_past[['Re_norm','Fre_norm','Mo_norm', 'Te_norm', 'AVG_o_norm', 'Pro_norm', 'Pur_norm']] = scaler.fit_transform(
    rfm_past[['Recency_log','Frequency_log','Monetary_log', 'Tenure_log', 'Avg_Order_Value_log', 'Product_Diversity_log'
    , 'Purchase_Regularity_log']]
)

rfm_past.head()

| โมเดล                  | แนะนำ           |
| ---------------------- | --------------- |
| Linear / Ridge / Lasso | Log + Standard  |
| Neural Network         | Log + MinMax    |
| KNN / SVR              | Log + MinMax    |
| Tree-based (RF, XGB)   | ❌ ไม่ต้อง scale |
| RFM / Clustering       | Log + MinMax    |

Min-Max “ย่อช่วง” แต่ไม่แก้รูปทรงข้อมูล

log1p “เปลี่ยนรูปทรงข้อมูล” แต่ไม่ fix ช่วงค่า

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

skew_raw = rfm_past["Monetary"].skew()
skew_log = rfm_past["Monetary_log"].skew()
skew_min = rfm_past["Mo_norm"].skew()

# --- สร้างกราฟ (Histogram + KDE) ---
fig, axes = plt.subplots(1, 3, figsize=(16, 6))

# Plot 1: Before Preprocessing (Original Data)
sns.histplot(rfm_past["Monetary"], kde=True, ax=axes[0], color="red", bins=30)
axes[0].set_title(f"Before: Original Distribution\n(Skewness = {skew_raw:.2f})", fontsize=14, fontweight='bold')
axes[0].set_xlabel("Monetary (Raw)")
axes[0].set_ylabel("Sales")

# Plot 2: After Preprocessing (Log Transformed)
sns.histplot(rfm_past["Monetary_log"], kde=True, ax=axes[1], color="blue", bins=30)
axes[1].set_title(f"After: Log Transformation (log1p)\n(Skewness = {skew_log:.2f})", fontsize=14, fontweight='bold')
axes[1].set_xlabel("Log(Monetary + 1)")
axes[1].set_ylabel("Sales")

# Plot 3: After Preprocessing (Min-Max Transformed)
sns.histplot(rfm_past["Mo_norm"], kde=True, ax=axes[2], color="green", bins=30)
axes[2].set_title(f"After: Min-Max Transformation (Min-Max)\n(Skewness = {skew_min:.2f})", fontsize=14, fontweight='bold')
axes[2].set_xlabel("Min-Max(Monetary)")
axes[2].set_ylabel("Sales")

plt.tight_layout()
plt.show()

(The Problem):
 ข้อมูล Feature สำคัญอย่าง Monetary ก็มีความเบ้สูงมาก (Skewness = 14.94) และมีลักษณะหางยาว (Long Tail) จากกลุ่มลูกค้าที่ซื้อซ้ำเยอะผิดปกติ

(The Solution):ใช้เทคนิค Log Transformation (log1p) เพื่อลดความเบ้

ผลลัพธ์ของ Target: Skewness ลดลงเหลือ 1.08 ทำให้เห็นการกระจายตัวของกลุ่มลูกค้าชัดเจนขึ้น

ผลลัพธ์ของ Feature (Monetary): Skewness ลดลงเหลือ 0.01 กลายเป็นการกระจายตัวแบบระฆังคว่ำ (Normal Distribution) ที่สมบูรณ์แบบ

การค้นพบ: การทำ Min-Max Scaling ต่อจาก Log ไม่ได้เปลี่ยนรูปทรงของข้อมูล (Skewness คงที่ที่ 1.08) เพียงแต่เปลี่ยนสเกลแกน X ให้อยู่ในช่วง 0-1 เท่านั้น

_____________
Log ใช้แก้ "ทรง" (Shape/Distribution) ส่วน Min-Max ใช้แก้ "สเกล" (Scale/Range)

ถ้าทำ Min-Max บนข้อมูลที่เบ้มากๆ (Skewed) โดย ไม่ทำ Log ก่อน:

ค่า Outlier (เช่น ลูกค้าที่ซื้อ 1 ล้านบาท) จะถูกแปลงเป็น 1.0

ลูกค้าส่วนใหญ่ (ที่ซื้อ 100-500 บาท) จะถูกบีบอัดไปอยู่ที่ 0.0001 - 0.0005

ผลเสีย: ความแตกต่างระหว่างลูกค้าปกติจะหายไปหมดเลยครับ เพราะสเกลถูกดึงไปให้ Outlier คนเดียว

เมื่อทำ Log ก่อน: มันจะดึงค่า Outlier ลงมาให้ใกล้เกณฑ์ปกติก่อน (แก้ทรงกราฟให้สวย) แล้วค่อยทำ Min-Max ทีหลังเพื่อให้ข้อมูลกระจายตัวเต็มช่วง 0-1 อย่างเหมาะสม

ถ้าทำแค่ Log อย่างเดียว:

Log(Recency) อาจจะได้ค่าประมาณ 0 - 5.9

Log(Monetary) อาจจะได้ค่าประมาณ 4.6 - 13.8

ผลเสีย: ตัวเลขของ Monetary (13.8) ยังดู "ใหญ่กว่า" Recency (5.9) อยู่ดี ถ้าเอาไปเข้าโมเดลที่วัดระยะทาง (เช่น K-Means Clustering) โมเดลจะให้ความสำคัญกับ Monetary มากกว่า Recency โดยอัตโนมัติ ทั้งที่จริงๆ มันอาจจะสำคัญเท่ากัน

การทำ Min-Max ต่อท้าย: จะบีบให้ ทั้งคู่ อยู่ในช่วง 0 ถึง 1 เหมือนกันเป๊ะ ทำให้โมเดลมองทั้งสองปัจจัยอย่างยุติธรรมครับ

In [ ]:
df_rfm =rfm_past[['Re_norm','Fre_norm','Mo_norm', 'Te_norm', 'AVG_o_norm', 'Pro_norm', 'Pur_norm','Repurchase_Count']]
df_rfm.sort_values(by='Repurchase_Count', ascending=False)

ตัวแปรต้น (Features - X):

✅ เลือกใช้: ข้อมูลที่ผ่านการ Normalize แล้ว (Re_norm, Fre_norm, Mo_norm, ฯลฯ)

เหตุผล: ข้อมูลมีความเสถียร (Scale ใกล้เคียงกัน) ช่วยให้โมเดลเรียนรู้ได้เร็วและเปรียบเทียบความสำคัญของตัวแปรได้ง่าย

ตัวแปรตาม (Target - Y):

✅ เลือกใช้: ค่าดิบ (Raw Data) จากคอลัมน์ Repurchase_Count (เช่น 135.0, 117.0, 0.0)

❌ ไม่เลือกใช้: ค่าที่แปลงเป็น 0-1 (Repur_norm)

In [ ]:
from sklearn.model_selection import train_test_split

X = df_rfm[['Re_norm','Fre_norm','Mo_norm', 'Te_norm','AVG_o_norm','Pro_norm','Pur_norm']]
y = df_rfm['Repurchase_Count']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

1. การทำข้อสอบคนละวิชา (Loss Function Mismatch) 📉

เมื่อใช้ข้อมูลดิบ: โมเดล (โดยเฉพาะถ้าใช้ Default objective) พยายามลดค่า MSE (Mean Squared Error) คือพยายามทำให้ "ส่วนต่างที่เป็นตัวเงิน/จำนวนจริง" น้อยที่สุด

ตัวอย่าง: ทำนายผิดไป 10 ครั้ง คือเรื่องใหญ่ โมเดลจะพยายามแก้ตรงนี้

เมื่อทำ Log: คุณกำลังบังคับให้โมเดลลดค่า MSLE (Mean Squared Log Error) หรือ "ความคลาดเคลื่อนเชิงสัดส่วน" (Relative Error)

ตัวอย่าง: การทายผิดระหว่าง 10 เป็น 20 (ต่างกัน 100%) โมเดลจะมองว่ามีความผิด เท่ากันกับ การทายผิดระหว่าง 1,000 เป็น 2,000

ผลลัพธ์: โมเดลจะเลิกสนใจ "ลูกค้า รายใหญ่ (Whales)" ที่ค่า Repurchase_Count สูงๆ เพราะในมุมมอง Log ค่าความผิดพลาดของคนกลุ่มนี้มันดู "เล็กน้อย" ไปแล้ว แต่พอคุณวัดผลด้วย R
2
  (ซึ่งแคร์ค่าความต่างจริงๆ) ความผิดพลาดของลูกค้ากลุ่มนี้จะถูกขยายกลับมาจนทำให้ Score พังครับ

2. การสูญเสีย "น้ำหนัก" ของข้อมูลสำคัญ (Compression of Signal) 🐋

ค่า R
2
  จะสูงได้ก็ต่อเมื่อโมเดลสามารถจับ Pattern ของข้อมูลที่มีความแปรปรวน (Variance) สูงๆ ได้ ซึ่งใน Dataset นี้คือกลุ่ม Outliers (คนที่ซื้อเยอะๆ)

การทำ Log จะบีบค่าของคนซื้อเยอะๆ (เช่น 100 ครั้ง) ให้เหลือค่านิดเดียว (เช่น 4.6) และไปใกล้เคียงกับคนซื้อน้อยๆ

Min-Max ยิ่งซ้ำเติมด้วยการบีบทุกอย่างลงไปในกล่อง 0-1 ทำให้ความแตกต่างระหว่าง "ลูกค้าชั้นดี" กับ "ลูกค้าทั่วไป" เหลือน้อยมากจนโมเดลแยกไม่ออก หรือมองว่าเป็น Noise ไปเลย

In [ ]:
!pip install scikeras[tensorflow]
!pip install catboost
#  !pip install xgboost
#  !pip install lightgbm

1. ถ้าใช้ Linear Regression หรือ Neural Network 📉

👉 "ควรทำ" (หรือต้องทำ)

เหตุผล: โมเดลพวกนี้ไวต่อ Outlier และตั้งสมมติฐานว่า Error ต้องมีการกระจายตัวแบบ Normal Distribution

ถ้า Target เบ้มากๆ โมเดลจะพังครับ การทำ Log จะช่วยให้แม่นยำขึ้นมาก

2. ถ้าใช้ Tree-based (LightGBM, XGBoost, Random Forest) 🌳

👉 "ไม่จำเป็นต้องทำ" (และห้ามทำ Min-Max กับ Target)

นี่คือเคสของคุณครับ! โมเดลพวกนี้ทำงานด้วยการ "ตัดแบ่งช่วง" (Splitting) ต่อให้ข้อมูลเบ้แค่ไหน มันก็แค่หาจุดตัดที่เหมาะสม

ทำไมห้ามทำ Min-Max กับ Target?:

Target คือสิ่งที่เราต้องการทำนายค่า "ความต่าง" (Variance)

ถ้าคุณไปทำ Min-Max คุณกำลัง "บีบอัดความต่าง" ของยอดขาย 100 บาท กับ 1,000,000 บาท ให้อยู่ในกล่องแคบๆ (เช่น 0.001 กับ 1.0)

LightGBM จะมองว่า "อ๋อ สองคนนี้ต่างกันแค่ 0.99 เอง ไม่ต้องไปสนใจมากก็ได้" -> นี่คือสาเหตุที่ R
2
  ร่วงครับ

In [ ]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

# -------------------------
# PyTorch NN Wrapper
# -------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.base import BaseEstimator, RegressorMixin
class TorchNNRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, input_dim, hidden1=16, hidden2=8, lr=0.01, epochs=200):
        self.input_dim = input_dim
        self.hidden1 = hidden1
        self.hidden2 = hidden2
        self.lr = lr
        self.epochs = epochs
        self.model = None

    def _build_model(self):
        model = nn.Sequential(
            nn.Linear(self.input_dim, self.hidden1),
            nn.ReLU(),
            nn.Linear(self.hidden1, self.hidden2),
            nn.ReLU(),
            nn.Linear(self.hidden2, 1)
        )
        return model

    def fit(self, X, y):
        X_tensor = torch.tensor(X.values, dtype=torch.float32)
        y_tensor = torch.tensor(y.values.reshape(-1,1), dtype=torch.float32)
        self.model = self._build_model()
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)

        for epoch in range(self.epochs):
            self.model.train()
            optimizer.zero_grad()
            outputs = self.model(X_tensor)
            loss = criterion(outputs, y_tensor)
            loss.backward()
            optimizer.step()
        return self

    def predict(self, X):
        self.model.eval()
        X_tensor = torch.tensor(X.values, dtype=torch.float32)
        with torch.no_grad():
            y_pred = self.model(X_tensor).numpy()
        return y_pred
# -------------------------
# Define Keras model builder
# -------------------------
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
def build_dense_nn(input_dim):
    model = Sequential([
        Dense(16, activation='relu', input_dim=input_dim),
        Dense(8, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model
# scikeras wrapper
keras_nn = KerasRegressor(
    model=build_dense_nn,
    model__input_dim=X_train.shape[1],
    epochs=200,
    batch_size=32,
    verbose=0 )
# =========================
# 2. Define models
# =========================
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.001),
    "RandomForest": RandomForestRegressor(
        n_estimators=200, random_state=42, n_jobs=-1
    ),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "SVR": Pipeline([
        ("scaler", StandardScaler()), ##SVR sensitive กับ scale → ต้อง scale อีกครั้งเพื่อความมั่นใจ
        ("svr", SVR(C=1000,
        epsilon=0.2,
        gamma=0.01,
        kernel="rbf"))
    ]),
    "PyTorchNN": TorchNNRegressor(input_dim=X_train.shape[1], epochs=200, lr=0.01) ,
    "XGBoost": XGBRegressor(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1),
    "TensorFlowNN": keras_nn,
    "LightGBM": LGBMRegressor(  ##from lightgbm import LGBMRegressor
        objective="poisson",
        learning_rate=0.03,
        n_estimators=2000,
        num_leaves=31,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        random_state=42,
        force_col_wise=True
    ),
    "CatBoost": CatBoostRegressor(  ##from catboost import CatBoostRegressor
        iterations=300,
        depth=4,
        learning_rate=0.05,
        verbose=0,         # ปิด print log
        random_seed=42
    )

}

# =========================
# 3. Train + Evaluate
# =========================
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    results.append({
        "Model": name,
        "MAE": mean_absolute_error(y_test, y_pred),
        "RMSE": mean_squared_error(y_test, y_pred),
        "R2": r2_score(y_test, y_pred)
    })
results_df = pd.DataFrame(results)

# =========================
# 4. Ranking
# =========================
results_df["Rank_MAE"] = results_df["MAE"].rank(method="min")
results_df["Rank_RMSE"] = results_df["RMSE"].rank(method="min")
results_df["Rank_R2"] = results_df["R2"].rank(ascending=False, method="min")

results_df["Overall_Rank"] = (
    results_df["Rank_MAE"] +
    results_df["Rank_RMSE"] +
    results_df["Rank_R2"]
)

results_df = results_df.sort_values("Overall_Rank")

# =========================
# 5. Export Excel
# =========================
# results_df.to_excel("regression_model_comparison.xlsx", index=False)

results_df

MAE (Mean Absolute Error): ค่าเฉลี่ยของความคลาดเคลื่อนระหว่างค่าจริงกับค่าที่โมเดลทำนาย, โมเดลทำนายพลาดประมาณ 1 ครั้งต่อคนโดยเฉลี่ย

RMSE (Root Mean Squared Error): ค่า error ที่ให้น้ำหนักกับความผิดพลาดขนาดใหญ่เป็นพิเศษ, RMSE = MAE เสมอ > แสดงว่ามีบางรายที่โมเดลพลาดมาก

R2 (R-squared): สัดส่วนของความแปรปรวนในตัวแปรเป้าหมายที่โมเดลสามารถอธิบายได้, โมเดลอธิบายพฤติกรรมการซื้อซ้ำได้ประมาณ 63%

| Dimension        | Regression      | Probability | Classification |
| ---------------- | --------------- | ----------- | -------------- |
| Output           | Count           | Probability | Class          |
| Loss             | MAE / RMSE      | Log Loss    | CE             |
| Business Meaning | Expected volume | Likelihood  | Yes / No       |
| Information      | สูง             | สูง         | ต่ำ            |
| Interpretability | กลาง            | สูง         | สูง            |
| Recommended      | ⭐⭐⭐⭐            | ⭐⭐⭐⭐⭐       | ⭐⭐             |


Linear kernel
C: 5 ค่า
epsilon: 5 ค่า
→ 25 models

RBF kernel
C: 5
epsilon: 5
gamma: 6
→ 150 models
รวมทั้งหมด = 175 models

cv=5
แต่ละ model ต้อง train 5 ครั้ง = 175 × 5 = 875 ครั้ง train

กลุ่ม Regressi
1. กลุ่ม Linear (เส้นตรง) 📏

เหมาะกับ: ข้อมูลที่มีความสัมพันธ์แบบ "ยิ่งมาก...ยิ่งมาก" หรือ "ยิ่งมาก...ยิ่งน้อย" อย่างชัดเจนและสม่ำเสมอ

ตัวอย่างโจทย์: การทำนาย "เงินเดือน (Salary)" จาก "ประสบการณ์ทำงาน (Years of Exp)"
ลักษณะข้อมูล:

X: ประสบการณ์ 1 ปี, 2 ปี, 5 ปี, 10 ปี
Y: เงินเดือน 20k, 25k, 50k, 90k
ทำไมต้องกลุ่มนี้: เพราะโดยทั่วไปยิ่งทำงานนาน เงินเดือนยิ่งขึ้นเป็นเส้นตรง (Trend ชัดเจน) ไม่ค่อยมีคนที่ทำงาน 10 ปีแล้วเงินเดือนลดเหลือ 5 พัน



2. กลุ่ม Regularized (Ridge, Lasso) 🛡️

เหมาะกับ: ข้อมูลที่มี "ตัวแปร (X) เยอะมากๆ" (หลักร้อยหรือหลักพัน) และหลายตัวอาจจะซ้ำซ้อนกัน

ตัวอย่างโจทย์: การทำนาย "ราคาบ้าน (House Price)" โดยมีข้อมูลสเปคบ้าน 200 คอลัมน์
ลักษณะข้อมูล:

X: ขนาดพื้นที่, จำนวนห้องนอน, จำนวนห้องน้ำ, ความกว้างโรงรถ, วัสดุปูพื้น, ระยะห่างจากโรงเรียน, ฯลฯ
Y: ราคาขายจริง
ทำไมต้องกลุ่มนี้: เพราะ "จำนวนห้องนอน" กับ "จำนวนห้องน้ำ" มักจะไปทิศทางเดียวกัน (บ้านหลังใหญ่ ห้องเยอะ ห้องน้ำก็เยอะ) ถ้าใช้ Linear ธรรมดา โมเดลจะงง (Multicollinearity) เราจึงต้องใช้ Lasso เพื่อตัดตัวแปรที่ซ้ำซ้อนทิ้ง หรือใช้ Ridge เพื่อลดความสำคัญมันลง



3. กลุ่ม GLM (Generalized Linear Models) 🎯

เหมาะกับ: ข้อมูลที่ "Target (Y) ไม่ใช่ตัวเลขปกติ" (เช่น เป็นจำนวนนับ, มีเลข 0 เยอะ, หรือเป็น % ความน่าจะเป็น)

ตัวอย่าง A (Poisson): ทำนาย "จำนวนอุบัติเหตุจราจรต่อวัน"

Y: 0, 1, 2, 5, 0, 1... (เป็นจำนวนเต็มบวกเสมอ ห้ามติดลบ)
ตัวอย่าง B (Tweedie) <-- เคสของคุณอยู่ตรงนี้!: ทำนาย "ยอดเคลมประกันภัย (Insurance Claim)" หรือ "ยอดซื้อซ้ำ (Repurchase)"

ลักษณะข้อมูล:

ลูกค้า 90% ไม่เคลมเลย (Y=0)
ลูกค้า 10% ที่เหลือ เคลมหนักมาก (Y=10,000 ถึง 1,000,000)
ทำไมต้องกลุ่มนี้: เพราะถ้าใช้ Linear ปกติ โมเดลจะพยายามลากเส้นผ่านเลข 0 จะทำให้ค่าทำนายติดลบ (เช่น ทำนายว่าเคลม -500 บาท) ซึ่งเป็นไปไม่ได้



4. กลุ่ม Non-Linear / Machine Learning (Trees & NN) 🌳🤖

เหมาะกับ: ข้อมูลที่มี "เงื่อนไขซับซ้อน" หรือ "ความสัมพันธ์แบบไม่ต่อเนื่อง" (ยึกยือ, เป็นขั้นบันได)

ตัวอย่างโจทย์: ทำนาย "การเลิกใช้บริการ (Churn)" หรือ "พฤติกรรมลูกค้า (RFM)"
ลักษณะข้อมูล:

คนอายุน้อย + รายได้น้อย = ซื้อเยอะ (วัยรุ่นชอบช้อป)
คนอายุเยอะ + รายได้น้อย = ซื้อน้อย (ประหยัด)
คนอายุเยอะ + รายได้เยอะ = ซื้อปานกลาง (ซื้อของแพงแต่นานๆ ที)
ทำไมต้องกลุ่มนี้ (เช่น LightGBM):

ความสัมพันธ์มันไม่ได้เป็นเส้นตรง (ไม่ใช่ว่า ยิ่งรวยยิ่งซื้อเยอะเสมอไป)
มันมี "จุดตัด" (Interaction) ระหว่าง อายุ กับ รายได้
Tree-based model สามารถสร้างกฎ: IF Age < 25 AND Income < 15k THEN Score = High ได้อย่างแม่นยำ



สรุปตำแหน่งโปรเจกต์ของคุณ 📍

ข้อมูล Repurchase Count ของคุณ:

หน้าตา (Distribution): เหมือน กลุ่ม 3 (Tweedie) คือมี 0 เยอะ และมีหางยาว
ความสัมพันธ์ (Relationship): มีความซับซ้อนเหมือน กลุ่ม 4 (Non-Linear) คือลูกค้าแต่ละกลุ่ม (RFM) มีพฤติกรรมไม่เหมือนกัน

ทางออกที่ดีที่สุด: จึงเป็นการใช้ Algorithm ของกลุ่ม 4 (LightGBM) แต่ตั้งค่า Objective ให้เลียนแบบ กลุ่ม 3 (objective='tweedie') นั่นเองครับ!

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# features / target
X_cb = df_rfm[["Re_norm","Fre_norm","Mo_norm","Te_norm","AVG_o_norm","Pro_norm","Pur_norm"]]
y_cb = df_rfm["Repurchase_Count"]

# stratify แบบ bin กัน y skew (optional)
y_bin = pd.qcut(y_cb.rank(method="first"), q=10, labels=False)

X_cb_train, X_cb_test, y_cb_train, y_cb_test = train_test_split(
    X_cb, y_cb, test_size=0.2, random_state=42, stratify=y_bin
)


In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold, cross_val_score
import numpy as np

cv = KFold(n_splits=5, shuffle=True, random_state=42)

# search space แบบในรูป
search_space = []
for depth in [4, 6, 8]:
    for lr in [0.01, 0.03, 0.05, 0.1]:
        for l2 in [1, 3, 5, 7, 9]:
            search_space.append((depth, lr, l2))

best = {"cv_r2": -999, "params": None}

for depth, lr, l2 in search_space:
    model = CatBoostRegressor(
        loss_function="RMSE",
        iterations=800,
        learning_rate=lr,
        depth=depth,
        l2_leaf_reg=l2,
        random_seed=42,
        verbose=0,
        allow_writing_files=False
    )

    scores = cross_val_score(model, X_cb_train, y_cb_train, cv=cv, scoring="r2")
    mean_r2 = scores.mean()

    if mean_r2 > best["cv_r2"]:
        best["cv_r2"] = mean_r2
        best["params"] = (depth, lr, l2)

best


## Finalize tuned CatBoost model
This section builds the final CatBoost model using the best params from CV, fits on the training split, and evaluates on the test split.


In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
from catboost import CatBoostRegressor

best_depth, best_lr, best_l2 = best["params"]

final_cb = CatBoostRegressor(
    loss_function="RMSE",
    iterations=2000,
    learning_rate=best_lr,
    depth=best_depth,
    l2_leaf_reg=best_l2,
    random_seed=42,
    verbose=0,
    allow_writing_files=False
)

final_cb.fit(X_cb_train, y_cb_train)

pred_test = final_cb.predict(X_cb_test)

print("BEST CV R2 :", best["cv_r2"])
print("TEST R2    :", r2_score(y_cb_test, pred_test))
print("TEST MAE   :", mean_absolute_error(y_cb_test, pred_test))
print("TEST RMSE  :", np.sqrt(mean_squared_error(y_cb_test, pred_test)))
print("BEST PARAMS:", best["params"])


In [ ]:
df_features = rfm_past[[
    "Customer ID",
    "Re_norm","Fre_norm","Mo_norm","Te_norm","AVG_o_norm","Pro_norm","Pur_norm",
    "Repurchase_Count"
]].copy()

df_features = df_features.rename(columns={"Customer ID":"Customer_ID"})
df_features["Customer_ID"] = df_features["Customer_ID"].astype("int64")


In [ ]:
df_features = rfm_past[
    ["Customer ID","Re_norm","Fre_norm","Mo_norm","Te_norm","AVG_o_norm","Pro_norm","Pur_norm","Repurchase_Count"]
].copy()

df_features = df_features.rename(columns={"Customer ID":"Customer_ID"})
df_features["Customer_ID"] = df_features["Customer_ID"].astype("int64")


## Production decision flow (Customer_ID → prediction → segment)
From here onward, we use the trained `final_cb` model to predict for **all customers**, create ranks/segments, and prepare UI-ready lookup by `Customer_ID`.


In [ ]:
feature_cols = ["Re_norm","Fre_norm","Mo_norm","Te_norm","AVG_o_norm","Pro_norm","Pur_norm"]
df_features["Pred_Repurchase_Count"] = final_cb.predict(df_features[feature_cols])


In [ ]:
import pandas as pd

df_features["Pred_Repurchase_Rank"] = df_features["Pred_Repurchase_Count"].rank(pct=True)

df_features["Segment"] = pd.qcut(
    df_features["Pred_Repurchase_Rank"],
    q=3,
    labels=["Low","Medium","High"]
)


In [ ]:
df_ui = df_features.set_index("Customer_ID", drop=False)

cid = df_ui["Customer_ID"].iloc[0]  # ลองหยิบคนแรก
df_ui.loc[cid, ["Segment","Pred_Repurchase_Count","Repurchase_Count"]]


In [ ]:
df_rfm = df_rfm.copy()


In [ ]:
import pandas as pd

importance = pd.Series(
    final_cb.get_feature_importance(),
    index=feature_cols
).sort_values(ascending=False)

importance


In [ ]:
import matplotlib.pyplot as plt

importance.plot(kind="bar")
plt.title("CatBoost Feature Importance")
plt.xlabel("Feature")
plt.ylabel("Importance")
plt.show()


## Optional experiments below (can ignore)
These cells are older/alternative approaches (e.g., threshold-based segments). Keep for reference, but you don’t need them for the main flow.


In [ ]:
col = "Pred_Repurchase_Count"

q40 = df_features[col].quantile(0.40)
q70 = df_features[col].quantile(0.70)

df_features["Segment"] = "Low"
df_features.loc[df_features[col] >= q40, "Segment"] = "Medium"
df_features.loc[df_features[col] >= q70, "Segment"] = "High"


In [ ]:
print(df_features.columns)
print(df_ui.columns)


In [ ]:
df_ui = df_features.set_index("Customer_ID", drop=False).copy()


In [ ]:
cid = int(df_ui["Customer_ID"].iloc[0])
df_ui.loc[cid, ["Segment","Pred_Repurchase_Count","Repurchase_Count"]]


In [ ]:
def available_options(segment):
    if segment == "High":
        return ["Cashback", "Gift", "Lucky Draw", "Save Points"]
    elif segment == "Medium":
        return ["Gift", "Lucky Draw", "Save Points"]
    else:
        return ["Save Points", "Lucky Draw"]


In [ ]:
def customer_screen(customer_id: int):
    if customer_id not in df_ui.index:
        return {"status":"not_member", "message":"กรุณาสมัครสมาชิก"}

    row = df_ui.loc[customer_id]
    seg = str(row["Segment"])
    return {
        "status": "ok",
        "Customer_ID": int(row["Customer_ID"]),
        "Segment": seg,
        "Pred_Repurchase_Count": float(row["Pred_Repurchase_Count"]),
        "Options": available_options(seg)
    }


In [ ]:
cid = int(df_ui["Customer_ID"].iloc[0])
customer_screen(cid)


In [ ]:
import pandas as pd

# ตารางบันทึกการเลือก (จำลอง DB)
df_choice_log = pd.DataFrame(
    columns=["Customer_ID", "Segment", "Selected_Option", "Pred_Repurchase_Count"]
)

def submit_choice(customer_id: int, selected_option: str):
    if customer_id not in df_ui.index:
        return {"status": "not_member"}

    row = df_ui.loc[customer_id]
    seg = str(row["Segment"])
    allowed = available_options(seg)

    if selected_option not in allowed:
        return {
            "status": "invalid_option",
            "allowed_options": allowed
        }

    global df_choice_log
    df_choice_log = pd.concat(
        [
            df_choice_log,
            pd.DataFrame([{
                "Customer_ID": customer_id,
                "Segment": seg,
                "Selected_Option": selected_option,
                "Pred_Repurchase_Count": row["Pred_Repurchase_Count"]
            }])
        ],
        ignore_index=True
    )

    return {"status": "saved"}


In [ ]:
submit_choice(cid, "Save Points")


In [ ]:
df_choice_log


In [ ]:
def run_customer_flow(customer_id: int, selected_option: str | None = None):
    # 1) โชว์หน้าจอ (segment + options)
    screen = customer_screen(customer_id)
    if screen["status"] != "ok":
        return screen

    # 2) ถ้ายังไม่กดอะไร ให้คืนหน้าจอไปก่อน
    if selected_option is None:
        return screen

    # 3) ถ้ากดแล้ว ให้บันทึก
    return submit_choice(customer_id, selected_option)


In [ ]:
run_customer_flow(cid)


In [ ]:
run_customer_flow(cid, "Lucky Draw")


In [ ]:
df_choice_log.tail(5)


In [ ]:
def submit_choice(customer_id: int, selected_option: str):
    if customer_id not in df_ui.index:
        return {"status": "not_member"}

    row = df_ui.loc[customer_id]
    seg = str(row["Segment"])
    allowed = available_options(seg)

    if selected_option not in allowed:
        return {"status": "invalid_option", "allowed_options": allowed}

    global df_choice_log

    # ลบ record เก่าของลูกค้าคนนี้ออกก่อน (กันซ้ำ)
    df_choice_log = df_choice_log[df_choice_log["Customer_ID"] != customer_id].copy()

    # ใส่ record ใหม่ (ล่าสุด)
    df_choice_log = pd.concat(
        [
            df_choice_log,
            pd.DataFrame([{
                "Customer_ID": customer_id,
                "Segment": seg,
                "Selected_Option": selected_option,
                "Pred_Repurchase_Count": float(row["Pred_Repurchase_Count"])
            }])
        ],
        ignore_index=True
    )

    return {"status": "saved"}


In [ ]:
run_customer_flow(cid, "Save Points")
run_customer_flow(cid, "Lucky Draw")
df_choice_log


In [ ]:
def simulate_ui():
    raw = input("กรอก Customer_ID: ").strip()
    if not raw.isdigit():
        print("กรอกเป็นตัวเลขเท่านั้น")
        return

    customer_id = int(raw)

    screen = run_customer_flow(customer_id)
    if screen["status"] != "ok":
        print(screen["message"])
        return

    print("\n--- ผลลูกค้า ---")
    print("Customer_ID:", screen["Customer_ID"])
    print("Segment:", screen["Segment"])
    print("Pred_Repurchase_Count:", round(screen["Pred_Repurchase_Count"], 6))

    options = screen["Options"]
    print("\n--- เลือกสิทธิ์ ---")
    for i, opt in enumerate(options, 1):
        print(f"{i}. {opt}")

    pick = input("พิมพ์เลขตัวเลือกเพื่อบันทึก: ").strip()
    if not pick.isdigit():
        print("กรอกเป็นตัวเลขเท่านั้น")
        return

    idx = int(pick) - 1
    if idx < 0 or idx >= len(options):
        print("เลือกไม่ถูกต้อง")
        return

    selected = options[idx]
    res = run_customer_flow(customer_id, selected)
    print("\nผลบันทึก:", res["status"])
    print("เลือก:", selected)


In [ ]:
simulate_ui()


In [ ]:
df_choice_log.tail(10)


In [ ]:
# สำหรับระบบ (UI / Database)
ui_cols = [
    "Customer_ID",
    "Segment",
    "Pred_Repurchase_Count"
]

df_ui_final = df_features[ui_cols].copy()
df_ui_final.head()


In [ ]:
def api_get_customer(customer_id: int):
    try:
        screen = run_customer_flow(customer_id)
        return screen
    except Exception as e:
        return {"status": "error", "message": str(e)}


In [ ]:
api_get_customer(12346)


In [ ]:
{
  "status": "ok",
  "Customer_ID": 12346,
  "Segment": "Low",
  "Pred_Repurchase_Count": 0.480545,
  "Options": ["Save Points", "Lucky Draw"]
}


| Segment    | ความหมาย                                        |
| ---------- | --------------------------------------------------------- |
| **High**   | ลูกค้าที่โมเดลประเมินว่ามีแนวโน้มซื้อซ้ำ *สูงกว่าคนอื่น*  |
| **Medium** | ลูกค้าที่อยู่ระดับกลางของตลาด                             |
| **Low**    | ลูกค้าที่มีแนวโน้มซื้อซ้ำต่ำ *เมื่อเทียบกับลูกค้าทั้งหมด* |


1: Repurchase_Score ถูกสร้างจาก feature ชุดเดียวกัน
Repurchase_Score ถูกนิยามจากตัวชี้วัดเชิงพฤติกรรมลูกค้า เช่น Recency, Frequency และ Monetary (RFM)
ในขณะที่ตัวแปรอิสระ (X) ที่ใช้ในการ train โมเดล ประกอบด้วย RFM และ feature ที่แตกออกมาจาก RFM เดียวกัน เช่น Tenure, Average Order Value และ Purchase Regularity
ดังนั้น ความสัมพันธ์ระหว่าง X และ y จึง ไม่ใช่ black-box relationship แต่เป็นความสัมพันธ์เชิงกำหนด (quasi-deterministic)
โมเดลจึงทำหน้าที่เรียนรู้และประมาณ สูตรการคำนวณคะแนนเดิมกลับคืนมา

* Black-box:
ใส่ X เข้าไป → ได้ y ออกมา
แต่ไม่รู้สูตร ไม่รู้เหตุผลแน่ชัด
* Non black-box (หรือ deterministic):
รู้สูตร / โครงสร้างชัด
X ถูกนำไปคำนวณ y โดยตรง

2: ไม่มี noise แบบโลกจริง (Real-world noise)
ข้อมูลที่ใช้ไม่มีปัจจัยรบกวนภายนอก เช่น
ฤดูกาล (seasonality)
แคมเปญการตลาด (marketing shock)
การเปลี่ยนพฤติกรรมลูกค้าในระยะยาว (customer drift)
การไม่มี noise ทำให้โครงสร้างของข้อมูลมีความเสถียรสูง
ส่งผลให้โมเดลสามารถทำนายได้อย่างแม่นยำเกือบสมบูรณ์ทั้งใน train, test และ cross-validation

3: Normalization ร่วมกับ Tree-based / Boosting Models
การใช้ Min-Max Normalization ทำให้ทุก feature อยู่ในช่วงเดียวกัน
ช่วยให้ decision boundary ของโมเดลชัดเจนมากขึ้น
เมื่อใช้ร่วมกับ tree-based และ boosting models เช่น Random Forest หรือ XGBoost
โมเดลสามารถเรียนรู้ pattern ที่เป็นเชิงเส้นและไม่เชิงเส้นได้อย่างง่ายดาย
จึงให้ค่าประสิทธิภาพที่สูงและเสถียร